# Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Personalize: experiment label and API key

These should be the parent and subfolder names on Google Drive

In [ ]:
exp_label = "your_experiment_name"
exp_spec = "your_experiment_specification"

In [ ]:
# This pipeline uses automatic speech-to-text software provided by https://www.assemblyai.com/
# Create an account and replace the code below with your personal API key.
api_key = 'your_api_key'

# Install packages

In [ ]:
pip install assemblyai

In [ ]:
pip install praat-textgrids

In [ ]:
pip install pydub

In [ ]:
pip install SpeechRecognition

# Functions

In [ ]:
# transcribe wav files and save transcriptions in a dictionary
def transcribe(api_key, filenames, wav_dir, output_dictionary_dir, language_code):
    import os
    import assemblyai as aai
    import pickle

    # your AssemblyAI API key
    aai.settings.api_key = api_key

    # set language
    config = aai.TranscriptionConfig(language_code=language_code)

    # create transcriber
    transcriber = aai.Transcriber(config=config)

    # initialize transcription dictionary
    transc_dict = {}

    for file in filenames:
        # load wav file
        filepath = os.path.join(wav_dir, file)

        # transcribe wav file
        transcript = transcriber.transcribe(filepath)

        if transcript.error:
            print(file, transcript.error)
            transc_dict[file] = 'not_recognized'
        else:
            # save transcription in dictionary
            transc_dict[file] = transcript.text  or 'not_recognized'
            print(f"{file}: {transcript.text}")

    # save the dictionary
    with open(os.path.join(output_dictionary_dir, "transcriptions.p"), "wb") as f:
        pickle.dump(transc_dict, f)

    # format final transcriptions
    final_transc = {os.path.splitext(key)[0]: val for key, val in transc_dict.items()}

    return final_transc

In [ ]:
# This function is adapted from: https://github.com/Masato-Nakamura-3/TransCloze
def chronset_results_to_dictionary(chronset_dir, output_onset_dir):
    import os
    import pandas as pd
    import pickle

    onset_dict = dict()
    for textf in [t for t in os.listdir(chronset_dir) if ".txt" in t]:
        onset_df = pd.read_table(os.path.join(chronset_dir, textf), names = ["wav", "rt"])
        temp_dict = dict(zip([os.path.splitext(w)[0] for w in onset_df["wav"]], onset_df["rt"] / 1000))
        onset_dict.update(temp_dict)

    # save the dictionary
    with open(os.path.join(output_onset_dir, "onset.p"), "wb") as f:
        pickle.dump(onset_dict, f)

    return onset_dict

In [ ]:
# This function is adapted from: https://github.com/Masato-Nakamura-3/TransCloze
# This function generates .TextGrid files for Praat
# You can optionally pass a dictionary for the onset and/or for the duration of the files
# The onset function is basically for Chronset
# You can also give it context info to help hand correction

# If the audio files have heterogeneous durations, add an argument of "inconsistent = 1"

# The function replaces the RT with 0.1 if the RT is 0 or shorter, or is NaN, producing an error message

def create_textgrids(dict_transcription, output_dir, wav_dir, onset = dict(), inconsistent = False):
    import os
    import textgrids
    import numpy as np
    from pydub import AudioSegment

    filenames = [os.path.splitext(i)[0] for i in os.listdir(wav_dir) if (".wav" in i)]

    for k in filenames:
        # Load the TextGrid template and edit it
        tg = textgrids.TextGrid(f"/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_raw/temp_note.TextGrid")

        # If the duration of the audio files are inconsistent, edit each TextGrid files to match the duration of the audio files
        if inconsistent == True:
            audio = AudioSegment.from_file(os.path.join(wav_dir, k + ".wav"))
            tg["words"][1].xmax = tg["words"][-1].xmin = audio.duration_seconds - 0.001

            if tg.xmax != audio.duration_seconds:
                tg.xmax = audio.duration_seconds

                for tier in tg.keys():
                    tg[tier][-1].xmax = audio.duration_seconds

        tg["words"][1].text = dict_transcription[k + ".wav"]

        if k in onset.keys():
            # If the onset is not detected or is negative, set 0.1
            if np.isnan(onset[k]) or onset[k] <= 0 :
                print("No onset for ", k)
                tg["words"][0].xmax = tg["words"][1].xmin = 0.1
            else:
                tg["words"][0].xmax = tg["words"][1].xmin = onset[k]

        item_id = k.split("_")[1]

        tg.write(os.path.join(output_dir, k + ".TextGrid"))

    print("All textgrid files generated.")

# Create folders and designate paths

In [ ]:
import os

# Define the base path
base_path = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign'

# List of subdirectories to create
directories = [
    'wav'
    'transc_output',
    'chronset_input',
    'chronset_results',
    'onset_output',
    'tg_raw'
]

for dir_name in directories:
    dir_path = os.path.join(base_path, dir_name)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        print(f"Directory '{dir_path}' created successfully.")
    else:
        print(f"Directory '{dir_path}' already exists.")

# Run automatic transcriptions

In [ ]:
import os
len(os.listdir(wav_dir))

In [ ]:
# List all files in the wav folder
wav_files = os.listdir(wav_dir)

# Dictionary to count occurrences of each unique ID
file_ids = {}

# Iterate through the files in the folder
for filename in wav_files:
    if filename.endswith('.wav'):
        # Extract the unique ID (last four letters after the last '_')
        unique_id = filename.split('_')[-1].split('.')[0]

        # Count the occurrence of each unique ID
        if unique_id in file_ids:
            file_ids[unique_id] += 1
        else:
            file_ids[unique_id] = 1

# Classify the IDs based on whether they have exactly 80 files
ids_with_80_files = [id_ for id_, count in file_ids.items() if count == 80]
ids_with_fewer_or_more_files = [id_ for id_, count in file_ids.items() if count != 80]

# Print the results
print("IDs with exactly 80 files:"), print(len(ids_with_80_files))
print(ids_with_80_files)

print("\nIDs with fewer or more than 80 files:"), print(len(ids_with_fewer_or_more_files))
print(ids_with_fewer_or_more_files)

In [ ]:
# Remove files with IDs that have fewer or more than 80 files
for filename in wav_files:
    if filename.endswith('.wav'):
        # Extract the unique ID (last four letters after the last '_')
        unique_id = filename.split('_')[-1].split('.')[0]

        # If the ID has fewer or more than 80 files, delete the file
        if unique_id in ids_with_fewer_or_more_files:
            file_path = os.path.join(wav_dir, filename)
            os.remove(file_path)
            print(f"Deleted: {filename}")

print("Files with IDs that have fewer or more than 80 files have been deleted.")

In [ ]:
len(os.listdir(wav_dir))

In [ ]:
# get list of wav file names
filenames = [w for w in os.listdir(wav_dir) if (".wav" in w)]

# generate transcriptions and store them in a dictionary
# replace "language_code" with your language code (e.g., American English : "en_us"); check language_code here: https://www.assemblyai.com/docs/getting-started/supported-languages
language_code = "en_us"
dict_transcription = transcribe(api_key, filenames, wav_dir, output_dictionary_dir, language_code)

In [ ]:
len(dict_transcription)

# Run script below if you only need speech onset data (get onset data using Chronset)

This section uses Chronset to get onset data and combines with with transcriptions

In [ ]:
import os
import zipfile
from pydub import AudioSegment
import speech_recognition as sr

# Set the name of the zip files
# The output files will be named as "{experiment_name}_{number}".zip (e.g. planet_cloze_1.zip)
zipname = f"{exp_label}_{exp_spec}_"

# # Check wav files for errors
# for f in filenames:
#     fsize = os.path.getsize(os.path.join(wav_dir, f))
#     if fsize < 100000:
#         print(f)
#         print(fsize)
#         with sr.AudioFile(os.path.join(wav_dir, f)) as wavf:
#             print(wavf.DURATION)

# Create batches of 500 wav files (i.e., maximum file counts Chronset can handle)
# Make lists of 500 or fewer wav files
fewerthan500 = []
temp = []
for f in filenames:
    temp.append(f)
    if len(temp) == 500:
        fewerthan500.append(temp)
        temp = []
if len(temp) > 0:
    fewerthan500.append(temp)

# Total nuber of wav files
len(filenames)
# Number of zip files
len(fewerthan500)

# Output zip files (with 500 or fewer wav files)
fnum = 0
for filelist in fewerthan500:
    fnum = fnum + 1
    with zipfile.ZipFile(os.path.join(chronset_input_dir, zipname+str(fnum) + ".zip"), 'w', compression=zipfile.ZIP_DEFLATED) as new_zip:
        for f in filelist:
            new_zip.write(os.path.join(wav_dir, f), arcname = f)

Go to https://www.bcbl.eu/databases/chronset and upload each zip file that is in the **chronset_input** folder.

Retrieve Chronset results from your mailbox and upload those files to the **chronset_results** folder.

In [ ]:
# get onsets and store them in a dictionary
import pickle
onset = chronset_results_to_dictionary(chronset_dir, output_onset_dir)
onset = pickle.load( open( f"/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/onset_output/onset.p", "rb" ) )

In [ ]:
# generate textgrid files and store them in tg_dir_final
import pickle
transcriptions = pickle.load( open( f"/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/transc_output/transcriptions.p", "rb" ) )
create_textgrids(transcriptions, tg_dir, wav_dir, onset, inconsistent = True)

No onset for  28_SP_bcob
No onset for  33_PP_hbvp
No onset for  36_SS_hbvp
No onset for  38_SP_kojh
No onset for  44_SS_hbvp
No onset for  4_masculine_mismatch_bcob
No onset for  58_unergative_bcob
No onset for  59_unergative_bcob
No onset for  64_unergative_bcob
No onset for  77_unaccusative_hbvp
No onset for  80_unaccusative_bcob
No onset for  7_masculine_mismatch_nzne
All textgrid files generated.


# Run script below if you need onset + alignment of multi-word utterances

This script does not produce onset data, only transcriptions.

In [ ]:
# generate textgrid files and store them in tg_dir_final
import pickle
transcriptions = pickle.load( open( f"/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/transc_output/transcriptions.p", "rb" ) )
create_textgrids(transcriptions, tg_dir, wav_dir, inconsistent = True)

All textgrid files generated.


In [ ]:
len(transcriptions)

1290

In [ ]:
import os
len(os.listdir(tg_dir))

1291

# Prepare files for running MFA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
len(os.listdir(tg_dir))

In [ ]:
# Define the source folder paths
wav_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/wav'
tg_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_raw'
wav_tg_comb = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/wav_tg_comb'

# Create the destination folder if it doesn't exist
if not os.path.exists(wav_tg_comb):
    os.makedirs(wav_tg_comb)

def copy_folder_contents(source, destination):
    import shutil
    for item in os.listdir(source):
        source_item = os.path.join(source, item)
        destination_item = os.path.join(destination, item)
        # If it's a file, copy it
        if os.path.isfile(source_item):
            shutil.copy(source_item, destination_item)
        # If it's a folder, recursively copy its contents
        elif os.path.isdir(source_item):
            if not os.path.exists(destination_item):
                os.makedirs(destination_item)
            copy_folder_contents(source_item, destination_item)

copy_folder_contents(wav_dir, wav_tg_comb)
copy_folder_contents(tg_dir, wav_tg_comb)

print("Files have been successfully copied to the new folder.")

In [ ]:
wav_tg_comb_backup = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/wav_tg_comb_backup'
if not os.path.exists(wav_tg_comb_backup):
    os.makedirs(wav_tg_comb_backup)
copy_folder_contents(wav_tg_comb, wav_tg_comb_backup)
print("Folder has been successfully duplicated.")

In [ ]:
len(os.listdir(wav_dir)), len(os.listdir(tg_dir)), len(os.listdir(wav_tg_comb), len(os.listdir(wav_tg_comb_backup))

In [ ]:
import os
from shutil import move

# Set your source directory
wav_tg_comb = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/wav_tg_comb'
files_per_subfolder = 100  # Adjust the number of pairs you want per subfolder

# List all files and filter out non '.wav' and '.textgrid' files
all_files = os.listdir(wav_tg_comb)
wav_files = sorted([f for f in all_files if f.endswith('.wav')])
textgrid_files = sorted([f for f in all_files if f.endswith('.TextGrid')])

# Combine the files into pairs based on matching base filenames
pairs = []
for wav_file in wav_files:
    base_name = os.path.splitext(wav_file)[0]
    textgrid_file = base_name + '.TextGrid'
    if textgrid_file in textgrid_files:
        pairs.append((wav_file, textgrid_file))

In [ ]:
# Function to create subfolders and move files
def create_subfolders_and_move_files(pairs, source_directory, files_per_subfolder):
    subfolder_count = 1
    for i in range(0, len(pairs), files_per_subfolder):
        # Create a new subfolder in the source directory
        subfolder_path = os.path.join(source_directory, f'subfolder_{subfolder_count}')
        os.makedirs(subfolder_path, exist_ok=True)
        # Move a batch of pairs to the new subfolder
        for wav_file, textgrid_file in pairs[i:i + files_per_subfolder]:
            move(os.path.join(source_directory, wav_file), os.path.join(subfolder_path, wav_file))
            move(os.path.join(source_directory, textgrid_file), os.path.join(subfolder_path, textgrid_file))
        subfolder_count += 1
    print("Files have been successfully divided into subfolders.")

# Call the function
create_subfolders_and_move_files(pairs, wav_tg_comb, files_per_subfolder)

# RUN MFA

On your local computer, inside MFA folder (e.g., ~/Documents/MFA):
*   Create new folder and name it using the same format as in Google Drive (e.g., **PPP3_pilot**)
*   Upload **run_mfa_on_subsets.sh** file (from Github pipeline)

Inside the experiment folder (e.g., ~/Documents/MFA/PPP3_pilot):
*   Upload **wav_tg_comb** folder (from Google Drive)
*   Create new folder and name it **MFA_output**

Open **run_mfa_on_subsets.sh** file with text editor:
*   Change file paths to your experiment folders, save and close

Open terminal and run each code below line by line:
*   conda activate aligner
*   cd ~/Documents/MFA
*   chmod +x run_mfa_on_subsets.sh
*   ./run_mfa_on_subsets.sh

Upload **MFA_output** folder to your experiment folder on Google Drive


# Combine files after running MFA

Upload MFA-processed results folder to Google Drive and rename it: MFA_processed

In [ ]:
parent_directory = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/MFA_output'
# parent_directory = f'/Users/rosalee/Documents/MFA/{exp_label}/{exp_spec}_output'

destination_directory = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_MFA_processed'
# destination_directory = f'/Users/rosalee/Documents/MFA/{exp_label}/{exp_spec}_output_comb'

import os
os.makedirs(destination_directory, exist_ok=True)  # Create the destination directory if it doesn't exist

def copy_textgrid_files(source_dir, dest_dir):
    from shutil import copy2
    for root, dirs, files in os.walk(source_dir):
        for file in files:
            if file.endswith('.TextGrid'):
                source_file_path = os.path.join(root, file)
                destination_file_path = os.path.join(dest_dir, file)

                # Check if a file with the same name already exists in the destination directory
                if os.path.exists(destination_file_path):
                    # If so, create a new file name to avoid overwriting
                    base, extension = os.path.splitext(file)
                    counter = 1
                    new_file_name = f"{base}_{counter}{extension}"
                    destination_file_path = os.path.join(dest_dir, new_file_name)

                    # Increment the counter until a unique name is found
                    while os.path.exists(destination_file_path):
                        counter += 1
                        new_file_name = f"{base}_{counter}{extension}"
                        destination_file_path = os.path.join(dest_dir, new_file_name)

                # Copy the file
                copy2(source_file_path, destination_file_path)

    print("All TextGrid files have been copied to the destination folder: tg_MFA_processed.")

# Call the function
copy_textgrid_files(parent_directory, destination_directory)

All TextGrid files have been copied to the destination folder: tg_MFA_processed.


In [ ]:
# Check number of tg files in pre-MFA (tg_raw) & post-MFA (tg_MFA_processed) folders
tg_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_raw'
tg_MFA_processed_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_MFA_processed'

len(os.listdir(tg_dir)), len(os.listdir(tg_MFA_processed_dir))

(1291, 1204)

In [ ]:
# import os

# def list_files_not_in_other_folder(source_dir, comparison_dir):
#     source_files = set(os.listdir(source_dir))
#     comparison_files = set(os.listdir(comparison_dir))

#     # Find files that are in source_dir but not in comparison_dir
#     files_not_in_comparison = source_files - comparison_files

#     if files_not_in_comparison:
#         print("Files in source but not in comparison directory:")
#         for file in files_not_in_comparison:
#             print(file)
#     else:
#         print("All files are present in both directories.")

# tg_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_raw'
# tg_MFA_processed_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_MFA_processed'

# list_files_not_in_other_folder(tg_dir, tg_MFA_processed_dir)

In [ ]:
import os
import shutil

def copy_files_not_in_other_folder(source_dir, comparison_dir, new_folder):
    # Create the new folder if it doesn't exist
    if not os.path.exists(new_folder):
        os.makedirs(new_folder)

    source_files = set(os.listdir(source_dir))
    comparison_files = set(os.listdir(comparison_dir))

    # Find files that are in source_dir but not in comparison_dir
    files_not_in_comparison = source_files - comparison_files

    if files_not_in_comparison:
        print("Copying files from source to new folder:")
        for file in files_not_in_comparison:
            source_file_path = os.path.join(source_dir, file)
            new_file_path = os.path.join(new_folder, file)
            # Move (or copy) the file to the new folder
            shutil.copy(source_file_path, new_file_path)
            print(f"Copied: {file}")
    else:
        print("All files are present in both directories, no files to move.")

tg_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_raw'
tg_MFA_processed_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_MFA_processed'
new_folder = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_to_process'

copy_files_not_in_other_folder(tg_dir, tg_MFA_processed_dir, new_folder)

In [ ]:
len(os.listdir(new_folder))

173

In [ ]:
import os
import shutil

def copy_matching_wav_files(wav_dir, textgrid_folder, new_wav_folder):
    # Create the new folder if it doesn't exist
    if not os.path.exists(new_wav_folder):
        os.makedirs(new_wav_folder)

    # Get the list of .TextGrid files (without extensions) in the textgrid_folder
    textgrid_files = {os.path.splitext(file)[0] for file in os.listdir(textgrid_folder) if file.endswith('.TextGrid')}

    # Get all .wav files from wav_dir and copy those that match the .TextGrid file names
    wav_files = [file for file in os.listdir(wav_dir) if file.endswith('.wav')]

    for wav_file in wav_files:
        wav_file_base = os.path.splitext(wav_file)[0]  # Get the name without extension
        if wav_file_base in textgrid_files:
            source_wav_path = os.path.join(wav_dir, wav_file)
            destination_wav_path = os.path.join(new_wav_folder, wav_file)
            shutil.copy(source_wav_path, destination_wav_path)
            print(f"Copied: {wav_file}")

    if not textgrid_files:
        print("No matching .wav files to copy.")

# Example usage
tg_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_raw'
tg_MFA_processed_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_MFA_processed'
new_folder = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_to_process'
wav_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/wav'
new_wav_folder = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/wav_to_process'

# Copy matching wav files
copy_matching_wav_files(wav_dir, new_folder, new_wav_folder)


In [ ]:
import os
import shutil

def copy_matching_wav_files(wav_dir, textgrid_folder):
    # Get the list of .TextGrid files (without extensions) in the textgrid_folder
    textgrid_files = {os.path.splitext(file)[0] for file in os.listdir(textgrid_folder) if file.endswith('.TextGrid')}

    # Get all .wav files from wav_dir and copy those that match the .TextGrid file names
    wav_files = [file for file in os.listdir(wav_dir) if file.endswith('.wav')]

    for wav_file in wav_files:
        wav_file_base = os.path.splitext(wav_file)[0]  # Get the name without extension
        if wav_file_base in textgrid_files:
            source_wav_path = os.path.join(wav_dir, wav_file)
            destination_wav_path = os.path.join(textgrid_folder, wav_file)  # Copy to textgrid_folder
            shutil.copy(source_wav_path, destination_wav_path)
            print(f"Copied: {wav_file}")

    if not textgrid_files:
        print("No matching .wav files to copy.")

# Example usage
tg_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_raw'
tg_MFA_processed_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_MFA_processed'
new_folder = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_to_process'
wav_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/wav'

# Copy matching wav files into the same folder as the TextGrid files
copy_matching_wav_files(wav_dir, new_folder)

Copied: 28_SP_bcob.wav
Copied: 29_SS_bcob.wav
Copied: 30_PP_bcob.wav
Copied: 2_feminine_mismatch_bcob.wav
Copied: 31_PS_bcob.wav
Copied: 32_SP_bcob.wav
Copied: 34_PP_bcob.wav
Copied: 33_SS_bcob.wav
Copied: 35_PS_bcob.wav
Copied: 36_SP_bcob.wav
Copied: 37_SS_bcob.wav
Copied: 38_SP_kojh.wav
Copied: 37_SS_rror.wav
Copied: 38_PP_rror.wav
Copied: 38_SP_jmgf.wav
Copied: 38_PP_bcob.wav
Copied: 3_feminine_match_bcob.wav
Copied: 39_PS_bcob.wav
Copied: 40_SP_bcob.wav
Copied: 42_PP_bcob.wav
Copied: 41_SS_bcob.wav
Copied: 43_SS_itng.wav
Copied: 43_PS_bcob.wav
Copied: 44_SP_bcob.wav
Copied: 46_PP_bcob.wav
Copied: 45_SS_bcob.wav
Copied: 48_SP_bcob.wav
Copied: 47_PS_bcob.wav
Copied: 49_SS_bcob.wav
Copied: 50_PP_bcob.wav
Copied: 4_masculine_mismatch_bcob.wav
Copied: 51_PS_bcob.wav
Copied: 52_SP_bcob.wav
Copied: 53_SS_bcob.wav
Copied: 54_PP_bcob.wav
Copied: 55_PS_bcob.wav
Copied: 57_unergative_bcob.wav
Copied: 56_SP_bcob.wav
Copied: 58_unergative_bcob.wav
Copied: 59_unergative_bcob.wav
Copied: 5_mascul

In [ ]:
len(os.listdir(new_folder))/2

86.5

In [ ]:
# # Function to copy files from tg_raw if they don't exist in MFA_output
# def copy_files_if_not_exists(source, destination):
#     import shutil
#     import os
#     for filename in os.listdir(source):
#         source_file = os.path.join(source, filename)
#         destination_file = os.path.join(destination, filename)
#         # Check if it's a file and if it doesn't already exist in MFA_output
#         if os.path.isfile(source_file):
#             if not os.path.exists(destination_file):
#                 shutil.copy(source_file, destination_file)
#                 print(f"Copied: {filename}")

# tg_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_raw'
# tg_MFA_processed = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_MFA_processed'
# copy_files_if_not_exists(tg_dir, tg_MFA_processed)

# print("Copy operation completed.")

In [ ]:
# Check number of tg files in pre-MFA (tg_raw) & post-MFA (tg_MFA_processed) folders
tg_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_raw'
tg_MFA_processed_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_MFA_processed'

len(os.listdir(tg_dir)), len(os.listdir(tg_MFA_processed_dir))

(1291, 1204)

# Generate results file

In [ ]:
import os
import io
import textgrids
import pandas as pd
import numpy as np

tg_MFA_processed_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_MFA_processed'

results = []
for tg_loc in [f for f in os.listdir(tg_MFA_processed_dir) if f.endswith(".TextGrid")]:
    tg_path = os.path.join(tg_MFA_processed_dir, tg_loc)
    filename = tg_loc.rsplit(".", 1)[0]
    item = filename.split("_")[0]
    item_type = "_".join(filename.split("_")[1:-1])
    subject_id = filename.split("_")[-1]

    try:
        tg = textgrids.TextGrid(tg_path)
        for word in tg['words']:
            if word.text.strip():
                results.append([filename, item, item_type, subject_id, word.text, word.xmin, word.xmax, word.dur])
    except Exception as e:
        print(f"Skipping file {tg_loc} due to error: {e}")
        # Insert a row indicating the file was not recognized due to an error
        results.append([filename, item, item_type, subject_id, "not_recognized", None, None, None])

df = pd.DataFrame(results, columns = ['filename', 'item', 'item_type', 'subject_id', 'word', 'rt_onset', 'rt_offset', 'rt_dur'])
df[['type', 'condition']] = pd.DataFrame(df['item_type'].apply(lambda x: x.split('_') if '_' in x else [x, None]).tolist(), index=df.index)
df['word'] = df['word'].apply(lambda x: x.strip().lower())
df['item'] = df['item'].astype(int)
df['word_order'] = df.groupby('filename')['word'].cumcount() + 1

df = df.reset_index(drop=True)
df

In [ ]:
df.subject_id.nunique()